In [ ]:
%matplotlib notebook
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
from scipy.sparse.linalg import eigs
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import time

In [ ]:
# Get coefficients for finite difference given some degree of derivative and a stencil:
# s is the stencil - points with respect to 0. e.g., central difference is -1,1 i.e. -dx,+dx. forward is 0,1 etc.
# deg is the degree of derivative. for example s = -1,0,1 and deg=2 will give f(xdx)+f(x-dx)-2f(x) ([1,-2,1])
# coeffs are the coeffecients such as d^deg/dx^deg ~ sum_i coeff[i]*f(x[i0+s[i]])
def finite_difference_coeffs(s,deg):    
    s = np.array(s)
    N_stencil = len(s)
    if deg>=N_stencil:
        print(f"Error! Need at least {deg+1} points for order {deg} derivative")
        coeffs=np.nan+np.zeros(s.shape)
    #Create Van Der Monde matrix
    S = np.zeros((N_stencil,N_stencil))
    for i in range(N_stencil):
        S[i,:] = s**i
    d = np.zeros((N_stencil,1))
    d[deg] = np.math.factorial(deg)
    coeffs = (np.linalg.inv(S) @ d).reshape(s.shape)
    return coeffs    

# Given a finite difference scheme with stencil s of derivative of degree deg and some boundary_conditions:
# Return a NxN matrix D that returns the derivative (finite difference) of an evenly spaced array of length N with dx=1.
# For example, deg=1, forward difference s= [0,1] and wall BT will return a matrix with 1 in the upper diagonal and
#-1 in the diagonal, meaning the matrix returns element i+1 minus element i (so last element is bad for fwd difference)
def stencil_to_diff_matrix(N,s,deg,boundary_conditions="walls"):
    N_stencil = len(s)
    coeffs = finite_difference_coeffs(s,deg)
    D = np.zeros([N,N])
    E = np.eye(N)
    for i in range(N_stencil):
        shift = s[i]
        S = np.roll(E,shift,axis=1)
        if boundary_conditions!="periodic":
            if shift>0:
                S = np.triu(S)
            elif shift<0:
                S = np.tril(S)
        D += S*coeffs[i]
    return D

#Return a NxNy X NxNy matrix that acts as the discrete laplacian on a FLATTENED grid of Nx X Ny of dimensions Lx X Ly
def discrete_laplacian(N_arr,L_arr):    
    Nx,Ny = N_arr
    Lx,Ly = L_arr
    
    dx = Lx / (Nx+1)
    dy = Ly / (Ny+1)
    
    #FORWARD STENCIL
    N_stencil = 2
    s = np.arange(N_stencil,dtype=int)
    deg = 1
    # create d/dx and d/dy linear operators
    Dx = stencil_to_diff_matrix(Nx,s,deg) / dx
    Dy = stencil_to_diff_matrix(Ny,s,deg) / dy
    
    D2x = -Dx.T @ Dx
    D2y = -Dy.T @ Dy
    L = sparse.kron(sparse.eye(Ny),D2x) + sparse.kron(D2y,sparse.eye(Nx))
    return L
    

In [ ]:
Nx,Ny = 128,128
Lx,Ly = 3,3 #so we sample [-1.5,1.5]
N_eigs = 100

fps = 240
N_samples = 10**4
dt = 0.01
c1 = np.array([160,0,0])
c2 = np.array([255,139,197])


def shape_fun(X,Y):
    F=(X**2+Y**2-1.3)**3 + X**2 * Y**3 # heart
    return F.flatten()
boundary = np.array([0.,np.nan])
def ind_fun(F):
    ind1 =  F<boundary[0]
    if np.isfinite(boundary[1]):
        ind2 =  F>boundary[1]
    else:
        ind2 = ind1
    ind = ind1*ind2
    return ind

x = np.linspace(-Lx/2,Lx/2,Nx)
y = np.linspace(-Ly/2,Ly/2,Ny)
X,Y = np.meshgrid(x,y)
F = shape_fun(X,Y)
ind = ind_fun(F)

L = discrete_laplacian([Nx,Ny],[Lx,Ly])
L=L[ind,:][:,ind] # this is where the magic happens! we enforce the laplacian to act only on the shape

eigenvalues, eigenmodes = eigs(L,k=N_eigs,which="SM")
eigenmodes = np.real(eigenmodes) # eigenmodes are real but we avoid numerical errors/warnings
k = np.sqrt(-np.real(eigenvalues)) #eigenvalues are real and negative but this is to avoid numerical errors!

init_conditions = eigenmodes[:,0]*0 
init_conditions[np.random.randint(len(init_conditions))] = 10. #6856
init_conditions[6856] = 10. #6856
amp = eigenmodes.T @ init_conditions
phas = 0*amp
freq = k*1. # c=1.

In [ ]:
Z0 = (np.nan+X).flatten()
Z = Z0.copy()
Z[ind] = eigenmodes[:,1]

vals = np.zeros((256,3))
vals[:, 0] = np.linspace(c1[0]/256., c2[0]/256., 256)
vals[:, 1] = np.linspace(c1[1]/256., c2[1]/256., 256)
vals[:, 2] = np.linspace(c1[2]/256., c2[2]/256., 256)
newcmp = ListedColormap(vals)

fig,ax=plt.subplots()
I=plt.imshow(Z.reshape([Nx,Ny]),extent= [-Lx/2,Lx/2,Ly/2,-Ly/2],cmap=newcmp) # for I.set_data
plt.contour(X,Y,F.reshape(X.shape),np.sort(boundary[np.isfinite(boundary)]),colors="k",linestyles="solid")
_=ax.set_axis_off()

In [ ]:
for n in range(N_samples):
    t = n*dt
    Z[ind]=eigenmodes @ (amp*np.cos(freq*t+phas))
    I.set_data(Z.reshape([Nx,Ny]))
    fig.canvas.draw()
    time.sleep(1/fps)
